In [2]:
import ee

ee.Authenticate()

ee.Initialize(project='mse-uhi')

In [8]:
# Check if the Earth Engine API is installed
!pip show earthengine-api

# If the Earth Engine API is not installed, install it
!pip install earthengine-api

Name: earthengine-api
Version: 0.1.401
Summary: Earth Engine Python API
Home-page: 
Author: 
Author-email: Google LLC <noreply@google.com>
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: google-api-python-client, google-auth, google-auth-httplib2, google-cloud-storage, httplib2, requests
Required-by: eerepr, geemap


In [3]:
# Export Images
Landsat8_image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_170052_20170108')
band_names = Landsat8_image.bandNames().getInfo()
Landsat8_data = Landsat8_image.select(band_names)

# Creating default region
# region = ee.Geometry.Rectangle(37.07, 11.50, 37.39, 11.82)

# Singapore's region
singapore = ee.Geometry.Rectangle(103.59, 1.13, 104.07, 1.47)

# Creating a region
# Define the coordinates of Singapore's 4 corner coordinates.
# singapore_ll = ee.Geometry.Point([103.6, 1.28])
# singapore_ur = ee.Geometry.Point([104.4, 1.46])

# Create a rectangle geometry using the lower-left and upper-right corners.
# singapore_rectangle = ee.Geometry.Rectangle(singapore_ll, singapore_ur)

In [52]:
# Define the region of interest with corner coordinates.
# singapore_coordinates = [[103.6, 1.28], [104.4, 1.28], [104.4, 1.46], [103.6, 1.46]]
# singapore_rectangle = ee.Geometry.Polygon([singapore_coordinates])

# Setup the export to Google Drive.
task = ee.batch.Export.image.toDrive(
    image=Landsat8,
    description='singapore_L8',
    folder='L8_Data',
    scale=30,
    region=singapore,
    fileFormat='GeoTIFF'
) # check out coordinate system, make sure it's consistent to avoid running into inconsistencies in coordinates/NaN values
task.start()

In [55]:
print(task.status())

{'state': 'COMPLETED', 'description': 'imagetoDrive_L8', 'priority': 100, 'creation_timestamp_ms': 1715591879877, 'update_timestamp_ms': 1715591931806, 'start_timestamp_ms': 1715591885248, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/16sjcvaB9JV3tSH_RQ4t2AOGJ22oGDhMf'], 'attempt': 1, 'batch_eecu_usage_seconds': 1.3530495166778564, 'id': 'NRXV43ZFKZRKEHHJM3KE46QE', 'name': 'projects/mse-uhi/operations/NRXV43ZFKZRKEHHJM3KE46QE'}


In [26]:
pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 33.4 MB/s eta 0:00:00


In [59]:
import rasterio
import pandas as pd
import numpy as np
from google.colab import drive

# Mount Google Drive
drive.mount("/content/drive")

# Path to the GeoTIFF file in your Google Drive
file_path = '/content/drive/My Drive/L8_Data/imagetoDrive_L8_default.tif'

# Open the exported GeoTIFF file.
with rasterio.open(file_path) as src:
    array = src.read(1)  # Read only the first band, assuming band 10 is exported
    nodata_value = src.nodatavals[0]

# Convert the array to a DataFrame
df = pd.DataFrame(array)
df = df.stack().reset_index(name='B10')
df.columns = ['row', 'col', 'B10']

# Replace NoData values if nodata_value is not None
if nodata_value is not None:
    df['B10'].replace(nodata_value, np.nan, inplace=True)

# Extract all columns not just B10, B10 gives information in Kelvin
# Especially information on cloud cover and pixel quality, because those columns help to give insights on
# temperature/data inconsistencies, append all columns on the right

print(df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
          row   col         B10
0           0     0  307.061890
1           0     1  306.789429
2           0     2  306.500824
3           0     3  306.193756
4           0     4  305.946198
...       ...   ...         ...
1391143  1187  1166  301.672150
1391144  1187  1167  302.197937
1391145  1187  1168  302.748932
1391146  1187  1169  303.313446
1391147  1187  1170  303.873077

[1391148 rows x 3 columns]
